In [1]:
# Libraries
# ==============================================================================
import pandas as pd
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from skforecast.ForecasterAutoreg import ForecasterAutoreg

In [2]:
# Data download
# ==============================================================================
url = ('https://raw.githubusercontent.com/arifpras/forecastingDemand/main/20230129_database.csv')
data = pd.read_csv(url, sep=',', header=0, names=['month', 'changes_avgincomingbid_sun', 'changes_avgincomingbid_sbn', 'changes_dpkbankumum', 'dv_bankindonesia', 'vix_index', 'bfcius_bberg'])

# Data preparation
# ==============================================================================
data = data.rename(columns={'month': 'date'})
data['date'] = pd.to_datetime(data['date'], format='%d/%m/%Y')
data = data.set_index('date')
data = data.rename(columns={'changes_avgincomingbid_sun': 'y'})
data = data.asfreq('MS')
data = data.sort_index()
data.head()

steps = 12
data_train = data.iloc[:-steps, :]
data_test  = data.iloc[-steps:, :]

In [3]:
# Create and fit forecaster
# ==============================================================================
forecaster = ForecasterAutoreg(
                 regressor = XGBRegressor(),
                 lags = 12
             )

forecaster.fit(y=data['y'], exog=data[['changes_dpkbankumum', 'vix_index']])
forecaster

ForecasterAutoreg 
Regressor: XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...) 
Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12] 
Transformer for y: None 
Transformer for exog: None 
Window size: 12 
Weight function included: False 
Exogenous included: True 
Type of exogenous variable: <class 'pandas.core.frame.

In [4]:
# Predict
# ==============================================================================
forecaster.predict(steps=12, exog=data_test[['changes_dpkbankumum', 'vix_index']])

2022-07-01     6.691400
2022-08-01    -0.621117
2022-09-01    -3.284583
2022-10-01    -2.806607
2022-11-01     2.552429
2022-12-01   -31.884857
2023-01-01    82.482704
2023-02-01   -27.269938
2023-03-01   -16.551195
2023-04-01    -8.900335
2023-05-01    -6.700576
2023-06-01    10.659480
Freq: MS, Name: pred, dtype: float64

In [5]:
# Predictors importance
# ==============================================================================
forecaster.get_feature_importance()

,feature,importance
0,lag_1,0.009192
1,lag_2,0.100512
2,lag_3,0.012616
3,lag_4,0.022227
4,lag_5,0.050116
5,lag_6,0.026362
6,lag_7,0.025777
7,lag_8,0.037794
8,lag_9,0.008627
9,lag_10,0.080355
